In [ ]:
# Import all the necessary lib
import numpy as np
import pandas as pd
import pickle
import os
from numpy.linalg import norm
from sklearn.metrics import accuracy_score



In [ ]:

def MAP_Adaptation(speaker_data_path):
    """Doing one round of EM Algorithm for Adapting the Speaker Data """
    speaker_data = pd.read_csv(speaker_data_path,sep=' ',header=None)
    # speaker_data = speaker_data.drop(columns=0,axis=1)


    ubm_model = pickle.load(open(f'gmm_speaker_ubm_{cluster_Size}_zmuv.pkl' ,'rb'))
    N,D = speaker_data.shape
    # print('N',N , 'D',D)
    K = int(args.K)
    # print(K)
    w_ubm = ubm_model[0]
    mu_ubm = ubm_model[1]
    sigma_ubm = ubm_model[2]

    """For Now Writing the E_Step Algorithm Code here and then changing the Code Structure of K-Means"""
    gamma_nk = np.zeros((N,K))
    for i in range(0,K):
    
        # gamma_nk[:,i] = np.log(w_ubm[i]) * mp.logpdf(speaker_data ,mu_ubm[i],sigma_ubm[i,:,:]).ravel()
        gamma_nk[:,i] = (w_ubm[i]) * mp.pdf(speaker_data ,mu_ubm[i],sigma_ubm[i,:,:]).ravel()


    N_k = np.sum(gamma_nk,axis=0)
    gamma_nk /= N_k




    mu_updated = []
    for i in range(K):
        mu_updated.append(np.dot(gamma_nk[:,i],speaker_data))   
    mu_upd = np.stack(mu_updated,axis=0)
    # print('mu_up',mu_upd.shape)
    
    r =16
    alpha_k = N_k/(N_k + r)
    # print('alpha_k_1',alpha_k.shape)

    alpha_k = alpha_k.reshape(-1, 1)

    mu_new = alpha_k*mu_upd+(1-alpha_k)*mu_ubm

    """It came to my notice that the mu values are not adapting
    Possibilities are:
                    1.alpha value is small and (1- alpha)value is large so the mu_new = mu_ubm   --->took ZMUV
                    2.Why is the alpha value small-->N-K-->gamma-->ubm-->all program is wrong :)
                    3.Gamma value was low in E_Step and M_step , used that value to calculate parameters
                    
            Fix     Took Zero Mean Unit Variance to bring the MFCC Feature Vectors and fixed all the problems 
                        including the Accuracy   """
    

        

    return mu_new , sigma_ubm

In [ ]:
"""RUN This ONCE

This combines all the train dataset into one single file for MAP Adaptation"""

train_dir_path = '/home/tenet/Documents/IITM/CS6300_Speech_Technology/CS22Z121/Assignment_6/mfcc_zeromean_unit_variance/mfcc_13_split_train'
adapted_train_speaker_dir = 'speaker_adapted_ubm_train_1024_zmuv'
# if not os.path.exists(adapted_train_speaker_dir):
#     os.makedirs(adapted_train_speaker_dir)

for train_folders in os.listdir(train_dir_path):
    comb = []
    if not os.path.exists(os.path.join(adapted_train_speaker_dir,train_folders)):  
        os.makedirs(os.path.join(adapted_train_speaker_dir,train_folders) )
    for train_files in os.listdir(os.path.join(train_dir_path,train_folders)):
        file_path = os.path.join(train_dir_path,train_folders,train_files)
        file_read = pd.read_csv(file_path,sep=' ',header=None)
        comb.append(file_read)
        # file_prefix = pathlib.Path(train_folders)
        # print(file_prefix)
    # print(np.array(comb).shape)    
    Y = pd.concat(comb)
    # Y.drop(columns=0,axis=1)

    # print(Y.shape)
    # # save_file_path = os.path.join(adapted_train_speaker_dir,train_folders)
    # # print(save_file_path)
    # filename = "{}.mfcc".format(train_folders)
    # Y.to_csv(filename,sep=' ',header=None)  
    output_filename = "{}.mfcc".format(train_folders)
    output_filepath = os.path.join(adapted_train_speaker_dir,train_folders,output_filename)
    np.savetxt(output_filepath,Y,delimiter=' ')      


In [ ]:
train_dir_path = '/home/tenet/Documents/IITM/CS6300_Speech_Technology/CS22Z121/Assignment_6/mfcc_with_cepstral_subtraction/mfcc_train'

adapted_train_speaker_dir = 'speaker_adapted_ubm_train_2048_zmuv_3itr'
if not os.path.exists(adapted_train_speaker_dir):
    os.makedirs(adapted_train_speaker_dir)
c = 0    
for train_files in sorted(os.listdir(train_dir_path)):

    speakerpath = os.path.join(train_dir_path,train_files)
    
    o_filename = "{}.pkl".format(train_files.split('.')[0])
    
    
    pickle.dump(MAP_Adaptation(speakerpath),open(os.path.join(adapted_train_speaker_dir,o_filename),'wb'))   
    c +=1
    print('itr:',c)

In [ ]:
test_dir_path = '/home/tenet/Documents/IITM/CS6300_Speech_Technology/CS22Z121/Assignment_6/mfcc_with_cepstral_subtraction/mfcc_13_split_test'

adapted_test_speaker_dir = 'speaker_adapted_ubm_test_2048_zmuv_3itr'
if not os.path.exists(adapted_test_speaker_dir):
    os.makedirs(adapted_test_speaker_dir)
c = 0
for test_folder in sorted(os.listdir(test_dir_path)):
    # print(test_folder)
    t = []
    for test_file in os.listdir(os.path.join(test_dir_path,test_folder)):
        t.append(test_file)
    fileselected = t[randrange(2)]

    
    speakerpath = os.path.join(test_dir_path,test_folder,fileselected)
    o_filename = "{}.pkl".format(test_folder)

    
    pickle.dump(MAP_Adaptation(speakerpath),open(os.path.join(adapted_test_speaker_dir,o_filename),'wb'))    
    c+=1
    print('itr :',c)
    

In [ ]:
def supervector(speaker_data_mean):
    """ 'K' Mixtures in the GMM of Dimension 'd' --> Supervector Dimension 'Kd'    """
    return speaker_data_mean.reshape(-1)


def cosine_similarity(x,y):
    return np.dot(x,y)/(norm(x)*norm(y))

def Eucledian_distance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

In [ ]:
test_set_dir = 'speaker_adapted_ubm_test_1024_zmuv'
train_set_dir = 'speaker_adapted_ubm_train_1024_zmuv'
pred_labels = []
for test_speakers in sorted(os.listdir(test_set_dir)):
    score = []
    path_test_speakers = os.path.join(test_set_dir,test_speakers)
    test_data = supervector(pickle.load(open(path_test_speakers,'rb'))[0])
    train_labels = []
    for train_speakers in sorted(os.listdir(train_set_dir)):
        path_train_speakers = os.path.join(train_set_dir,train_speakers)
        train_data = supervector(pickle.load(open(path_train_speakers,'rb'))[0])
        # score.append(cosine_similarity(test_data,train_data))
        score.append(Eucledian_distance(test_data,train_data))
        
        train_labels.append(train_speakers.split('.')[0])
    # idx = score.index(max(score))
    idx = score.index(min(score))

    pred_labels.append(train_labels[idx])    
  
accuracy = accuracy_score(train_labels, pred_labels)
print(accuracy)
